# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [21]:
# Dependencies and Setup
import hvplot.pandas as hvplot
import pandas as pd
import requests
# Import API key
from api_keys import geoapify_key


In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dilling,12.0500,29.6500,22.89,29,71,4.41,SD,1702880818
1,1,puerto ayora,-0.7393,-90.3518,23.55,88,78,3.37,EC,1702880727
2,2,talnakh,69.4865,88.3972,-16.53,77,100,7.02,RU,1702880745
3,3,ishigaki,24.3448,124.1572,24.07,64,75,5.66,JP,1702880746
4,4,rabaul,-4.1967,152.1721,28.69,75,73,4.39,PG,1702880818


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [37]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    c = "Humidity",
    geo = True,
    tiles = "EsriStreet",
    frame_width = 800,
    frame_height = 600,
    color = "City"
)

# Display the map
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
list(city_data_df)

['City_ID',
 'City',
 'Lat',
 'Lng',
 'Max Temp',
 'Humidity',
 'Cloudiness',
 'Wind Speed',
 'Country',
 'Date']

In [5]:
city_data_df["City_ID"].count()

596

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df.notnull()

# Drop any rows with null values
city_df_notnull = city_data_df.dropna()
# Display sample data
city_df_notnull.head(1)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dilling,12.05,29.65,22.89,29,71,4.41,SD,1702880818


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
selected_columns = ["City", "Country", "Lat", "Lng", "Humidity"]
hotel_df = city_df_notnull[selected_columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = "" #Must always add a value into the column " " is needed to be null values\

# Display sample data
hotel_df.head(5)

,City,Country,Lat,Lng,Humidity,Hotel Name
0,dilling,SD,12.0500,29.6500,29,
1,puerto ayora,EC,-0.7393,-90.3518,88,
2,talnakh,RU,69.4865,88.3972,77,
3,ishigaki,JP,24.3448,124.1572,64,
4,rabaul,PG,-4.1967,152.1721,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
conditions = "access"
limit = 20  

# Parameters dictionary
params = {
    "categories": categories,
    "conditions": conditions,
    "apiKey": geoapify_key,
    "limit": limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.head(20).iterrows(): # I had to do the .head(20) method on the dataframe because the limit paramter was not working
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    if "features" in name_address and name_address["features"]:
        try:
            # Assuming the structure of the response has a "name" field within "properties"
            hotel_name = name_address["features"][0]["properties"].get("name", "No hotel found")
            hotel_df.loc[index, "Hotel Name"] = hotel_name
        except (KeyError, IndexError):
            # If no hotel name is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    else:
        # If "features" is empty, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
         # Log the search results
    print(f"{row['City']}, {row['Country']} - nearest hotel: {row['Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
dilling, SD - nearest hotel: 
puerto ayora, EC - nearest hotel: 
talnakh, RU - nearest hotel: 
ishigaki, JP - nearest hotel: 
rabaul, PG - nearest hotel: 
kone, NC - nearest hotel: 
waitangi, NZ - nearest hotel: 
sorokyne, UA - nearest hotel: 
longyearbyen, SJ - nearest hotel: 
pacific grove, US - nearest hotel: 
itaituba, BR - nearest hotel: 
slavyanka, RU - nearest hotel: 
green river, US - nearest hotel: 
grytviken, GS - nearest hotel: 
port elizabeth, ZA - nearest hotel: 
blackmans bay, AU - nearest hotel: 
adamstown, PN - nearest hotel: 
tiksi, RU - nearest hotel: 
margaret river, AU - nearest hotel: 
port-aux-francais, TF - nearest hotel: 


,City,Country,Lat,Lng,Humidity,Hotel Name
0,dilling,SD,12.0500,29.6500,29,No hotel found
1,puerto ayora,EC,-0.7393,-90.3518,88,Hostal La Mirada De Solitario George
2,talnakh,RU,69.4865,88.3972,77,Талнах
3,ishigaki,JP,24.3448,124.1572,64,アートホテル石垣島
4,rabaul,PG,-4.1967,152.1721,75,Rabaul Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
# Configure the map plot
map_plot_2 = hotel_df.head(50).hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriStreet",
    frame_width = 800,
    frame_height = 600,
    color = "City",
    hover = True,
    hover_cols="all"
)
#Save hvPlot in html to open in local browser
hvplot.save(map_plot_1, '../output_data/City_HvPlot_Fig2.html')
# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,index,Country,Humidity,Hotel Name)